In [1]:
from pipeline import SettlementPipeline
from transaction_parser import StringTransactionInputParser

allTransactions = '''
group A B C D E F
A 40 D
A 10 E
B 30 F
B 30 E
C 70 E
'''

pipeline = SettlementPipeline(inputParser=StringTransactionInputParser)
pipeline.getSettlementTransactions(allTransactions)

2026-02-08 01:38:44.488India Standard Time 20460 18748 INFO D has to pay C 40.0
2026-02-08 01:38:44.489India Standard Time 20460 18748 INFO E has to pay B 60.0
2026-02-08 01:38:44.490India Standard Time 20460 18748 INFO E has to pay A 50.0
2026-02-08 01:38:44.490India Standard Time 20460 18748 INFO F has to pay C 30.0
